In [1]:
import os, sys
from pathlib import Path

# notebook folder is sibling of src
NOTEBOOK_DIR = Path(os.getcwd())
PROJECT_ROOT = NOTEBOOK_DIR.parent  # because notebook/ and src/ are siblings
sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("sys.path[0]:", sys.path[0])

from src.config import TrainConfig
from iterative_driver_voting import run_experiment_for_setting


PROJECT_ROOT: /mnt/c/Users/truon/learning/ptit/research/trung/M_10_01_2025/code_v2/project
sys.path[0]: /mnt/c/Users/truon/learning/ptit/research/trung/M_10_01_2025/code_v2/project


In [2]:
EXPERIMENT_PLAN  = [
    # {
    #     "noise_ratios": [0.8],
    #     # "alphas": [0.3, 0.2, 0.6, 0.8],
    #     # "alphas": [0, 0.2,0.5,  0.6, 0.8],
    #     "alphas": [0.8],
    #     # "alphas": [0.3],
    #     "modes": ["vote_match_noisy", "vote_relabel", "ema_hard"],
    # },
    # {
    #     "noise_ratios": [0.6],
    #     "alphas": [0, 0.2, 0.3, 0.5, 0.8],
    #     # "alphas": [0.6, 0.3, 0.2,  0.8],
    #     # "alphas": [0.6],
    #     "modes": ["vote_match_noisy", "vote_relabel", "ema_hard"],

    # },
    # {
    #     "noise_ratios": [0.4],
    #     # "alphas": [0.8 ,0.3, 0.2, 0.6],        
    #     "alphas": [0, 0.2, 0.3, 0.5, 0.6],
    #     # "alphas": [0.8],        
    #     "modes": ["vote_match_noisy", "vote_relabel", "ema_hard"],

    # },
    {
        "noise_ratios": [0.2],
        # "alphas": [0.2 , 0.3 , 0.6, 0.8],
        "alphas": [0 , 0.3, 0.5, 0.6, 0.8],
        # "alphas": [0.2],
        "modes": ["vote_match_noisy", "vote_relabel", "ema_hard"],

    },
    {
        "noise_ratios": [0.4],
        # "alphas": [0.8 ,0.3, 0.2, 0.6],        
        # "alphas": [0, 0.2, 0.3, 0.5, 0.6],
        "alphas": [0.6],
        # "alphas": [0.8],        
        # "modes": ["vote_match_noisy", "vote_relabel", "ema_hard"],
        "modes": ["ema_hard", "vote_match_noisy", "vote_relabel" ],

    },
]


In [3]:
from pathlib import Path
import os

config = TrainConfig()

# (optional) đặt exp_name riêng cho voting experiments
config.exp_name = "cifar10_iter_ema_voting_sweep"

# base dir for all runs
base_dir = Path(config.make_exp_dir())
print("BASE_DIR:", base_dir)

# voting hyperparams (giữ cố định để so sánh mode)
DEFAULT_VOTING = {
    "voting_k": 3,
    "voting_agree_min": 2,
    "voting_source": "pred_hat",   # or "pred_ema"
    "voting_start_iter": 2,
}

# config.max_iterations = 10
# config.max_epochs_per_iter = 1

for block in EXPERIMENT_PLAN:
    for noise_ratio in block["noise_ratios"]:
        csv_dir = Path(config.data_dir) / "csvs" / f"noise_{noise_ratio}"
        train_csv = str(csv_dir / "train.csv")
        val_csv   = str(csv_dir / "val.csv")
        test_csv  = str(csv_dir / "test.csv")

        if not Path(train_csv).exists():
            raise FileNotFoundError(f"Missing CSVs at: {csv_dir}")

        for alpha in block["alphas"]:
            for mode in block["modes"]:
                # mutate config for this run
                config.noise_ratio = float(noise_ratio)
                config.alpha = float(alpha)
                config.filter_mode = mode

                config.voting_k = int(DEFAULT_VOTING["voting_k"])
                config.voting_agree_min = int(DEFAULT_VOTING["voting_agree_min"])
                config.voting_source = str(DEFAULT_VOTING["voting_source"])
                config.voting_start_iter = int(DEFAULT_VOTING["voting_start_iter"])

                # exp_dir structure (report-friendly)
                config.exp_dir = str(
                    base_dir
                    / f"noise_{noise_ratio}"
                    / f"alpha_{alpha}"
                    / f"mode_{mode}"
                    / f"k{config.voting_k}_agree{config.voting_agree_min}_{config.voting_source}_start{config.voting_start_iter}"
                )
                os.makedirs(config.exp_dir, exist_ok=True)

                print("\n" + "="*120)
                print("RUN:", {"noise": noise_ratio, "alpha": alpha, "mode": mode, "exp_dir": config.exp_dir})
                print("="*120)

                run_experiment_for_setting(config, train_csv, val_csv, test_csv)


INFO:iterative_driver_voting:=== Iteration 0 | filter_mode=vote_match_noisy | alpha=0.0000 | noise=0.20 ===


BASE_DIR: outputs/cifar10_iter_ema_voting_sweep

RUN: {'noise': 0.2, 'alpha': 0, 'mode': 'vote_match_noisy', 'exp_dir': 'outputs/cifar10_iter_ema_voting_sweep/noise_0.2/alpha_0/mode_vote_match_noisy/k3_agree2_pred_hat_start2'}


INFO:iterative_driver_voting:Train samples used (iter 0): 45000/45000 (train_label_col=label_noisy)
INFO:src.model_utils:Loaded ResNet18 with 10 output classes on device cuda
/mnt/c/Users/truon/learning/ptit/research/trung/M_10_01_2025/code_v2/project/src/train_loop.py:37: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if use_amp and device.startswith("cuda") else None
Train:   0%|                                                                                                                                       | 0/176 [00:00<?, ?it/s]/mnt/c/Users/truon/learning/ptit/research/trung/M_10_01_2025/code_v2/project/src/train_loop.py:53: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
INFO:src.train_loop:Iter 0 Epoch 0: train_acc_noisy=0.3799 train_acc_orig=0.4597 v


RUN: {'noise': 0.2, 'alpha': 0, 'mode': 'vote_relabel', 'exp_dir': 'outputs/cifar10_iter_ema_voting_sweep/noise_0.2/alpha_0/mode_vote_relabel/k3_agree2_pred_hat_start2'}


INFO:src.model_utils:Loaded ResNet18 with 10 output classes on device cuda
/mnt/c/Users/truon/learning/ptit/research/trung/M_10_01_2025/code_v2/project/src/train_loop.py:37: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if use_amp and device.startswith("cuda") else None
Train:   0%|                                                                                                                                       | 0/176 [00:00<?, ?it/s]/mnt/c/Users/truon/learning/ptit/research/trung/M_10_01_2025/code_v2/project/src/train_loop.py:53: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
INFO:src.train_loop:Iter 0 Epoch 0: train_acc_noisy=0.3799 train_acc_orig=0.4597 val_acc_orig=0.7432 val_acc_noisy=0.6026                                                  
INFO:src.i

...

INFO:iterative_driver_voting:Train samples used (iter 0): 45000/45000 (train_label_col=label_noisy)
INFO:src.model_utils:Loaded ResNet18 with 10 output classes on device cuda
/mnt/c/Users/truon/learning/ptit/research/trung/M_10_01_2025/code_v2/project/src/train_loop.py:37: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if use_amp and device.startswith("cuda") else None
Train:   0%|                                                                                                                                       | 0/176 [00:00<?, ?it/s]/mnt/c/Users/truon/learning/ptit/research/trung/M_10_01_2025/code_v2/project/src/train_loop.py:53: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
INFO:src.train_loop:Iter 0 Epoch 0: train_acc_noisy=0.2728 train_acc_orig=0.4076 v


RUN: {'noise': 0.4, 'alpha': 0.6, 'mode': 'vote_match_noisy', 'exp_dir': 'outputs/cifar10_iter_ema_voting_sweep/noise_0.4/alpha_0.6/mode_vote_match_noisy/k3_agree2_pred_hat_start2'}


INFO:src.model_utils:Loaded ResNet18 with 10 output classes on device cuda
/mnt/c/Users/truon/learning/ptit/research/trung/M_10_01_2025/code_v2/project/src/train_loop.py:37: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if use_amp and device.startswith("cuda") else None
Train:   0%|                                                                                                                                       | 0/176 [00:00<?, ?it/s]/mnt/c/Users/truon/learning/ptit/research/trung/M_10_01_2025/code_v2/project/src/train_loop.py:53: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
INFO:src.train_loop:Iter 0 Epoch 0: train_acc_noisy=0.2728 train_acc_orig=0.4076 val_acc_orig=0.6640 val_acc_noisy=0.4206                                                  
INFO:src.i


RUN: {'noise': 0.4, 'alpha': 0.6, 'mode': 'vote_relabel', 'exp_dir': 'outputs/cifar10_iter_ema_voting_sweep/noise_0.4/alpha_0.6/mode_vote_relabel/k3_agree2_pred_hat_start2'}


INFO:iterative_driver_voting:Train samples used (iter 0): 45000/45000 (train_label_col=label_noisy)
INFO:src.model_utils:Loaded ResNet18 with 10 output classes on device cuda
/mnt/c/Users/truon/learning/ptit/research/trung/M_10_01_2025/code_v2/project/src/train_loop.py:37: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if use_amp and device.startswith("cuda") else None
Train:   0%|                                                                                                                                       | 0/176 [00:00<?, ?it/s]/mnt/c/Users/truon/learning/ptit/research/trung/M_10_01_2025/code_v2/project/src/train_loop.py:53: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
INFO:src.train_loop:Iter 0 Epoch 0: train_acc_noisy=0.2728 train_acc_orig=0.4076 v